In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fiona
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import numpy as np

In [2]:
dataDir = r'C:\\Users\\DUANYUEYUN\\Documents\\ArcGIS\\Projects\\GRID3\\Sub-Saharan_health_facilities.gdb'
fiona.listlayers(dataDir)

['ISS_sub_saharan', 'HDX_WHO_sub_saharan_health_facilities']

In [3]:
df = gpd.read_file(dataDir, driver='FileGDB', layer='ISS_sub_saharan')

In [4]:
pd.set_option('display.max_columns', None)
df.head()

,starttime,endtime,today,deviceid,date_of_visit,ist,countries,states,districts,wards,designation,name_of_facility_visited,name_of_facility_visited_other,type_of_facility_visited,priority_level,facility_fp_acs,joint_supervision,supervised_by_dsno,supervised_by_who,current_feed_back,surveillance_fp_training,case_def_copy_copy1,facility_fp_acs_copy,physiotherapy_unit,long,lat,F27,F28,F29,F_index,F_parent_table_name,F_parent_index,F_tags,F_notes,F_version,F_duration,F_submitted_by,F_xform_id,longitude,latitude,Weeknumbers,YearofActiveSurv,geometry
0,05/11/2019 11:18:13 am +01:00,05/11/2019 11:51:00 am +01:00,"Tuesday, 5 November 2019",8.654590e+14,"Tuesday, 5 November 2019",IST_WEST,ALGERIA,ALGER,ALGER,Annasser,DISTRICT_SURVEILLANCE_OFFICER,Polyclinique les annassers,None,CSI,M,Y,N,NOT_IN_THE_PAST_1_YEAR,NOT_IN_THE_PAST_1_YEAR,N,6_MONTHS,AFP MEASLES CEREBROSPINAL_MENINGITIS NNT,Y,N,36.735202,3.077582,103.400000,5.000,None,1.0,None,-1.0,None,None,2.018070e+11,1968.0,None,908.0,36.734880,3.077293,45.0,2019.0,POINT Z (3.07758 36.73520 0.00000)
1,05/11/2019 11:29:29 am +01:00,05/11/2019 12:07:23 pm +01:00,"Tuesday, 5 November 2019",3.528230e+14,"Tuesday, 5 November 2019",IST_WEST,ALGERIA,ALGER,ALGER,Cité des annassers,DISTRICT_SURVEILLANCE_OFFICER,Polyclinique Khalef abed azize,None,CSI,M,Y,N,1_2_WEEKS,NOT_IN_THE_PAST_1_YEAR,N,6_MONTHS,AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGI...,Y,N,36.735584,3.077514,163.373718,4.288,None,2.0,None,-1.0,None,None,2.018070e+11,2275.0,None,908.0,36.735530,3.077479,45.0,2019.0,POINT Z (3.07751 36.73558 0.00000)
2,05/11/2019 11:26:12 am +01:00,05/11/2019 3:03:24 pm +01:00,"Tuesday, 5 November 2019",3.532850e+14,"Tuesday, 5 November 2019",IST_WEST,ALGERIA,ALGER,ALGER,Annasser,DISTRICT_SURVEILLANCE_OFFICER,Polyclinique Khalef Abdelaziz,None,CSI,M,Y,Y,1_2_WEEKS,NOT_IN_THE_PAST_1_YEAR,Y,6_MONTHS,AFP MEASLES CEREBROSPINAL_MENINGITIS NNT,Y,N,36.735539,3.077475,163.792931,6.000,None,3.0,None,-1.0,None,None,2.018070e+11,13031.0,None,908.0,36.735410,3.077145,45.0,2019.0,POINT Z (3.07747 36.73554 0.00000)
3,18/10/2017 11:44:22 am +01:00,18/10/2017 12:07:06 pm +01:00,"Wednesday, 18 October 2017",3.557550e+14,"Wednesday, 18 October 2017",IST_CENTRAL,ANGOLA,BIE,CUNHINGA,PS Hualondo,WHO_STAFF,PS Hualondo,None,CSI,L,Y,Y,1_2_WEEKS,NOT_IN_THE_PAST_1_YEAR,N,MORE_THAN_2_YRS,AFP MEASLES YELLOW_FEVER NNT,NA,N,-12.040417,16.947519,1730.800049,5.000,None,42.0,None,-1.0,None,None,2.017090e+11,1365.0,None,866.0,-12.040341,16.947544,43.0,2017.0,POINT Z (16.94752 -12.04042 0.00000)
4,25/10/2017 10:49:30 am +01:00,25/10/2017 10:56:37 am +01:00,"Wednesday, 25 October 2017",3.557550e+14,"Wednesday, 25 October 2017",IST_CENTRAL,ANGOLA,LUNDA NORTE,CHITATO,Distrito do Dundo,REGIONAL_SURVEILLANCE_OFFICER,Hospital Provincial do Dundo,None,GENERAL_HOSP,H,Y,Y,1_2_WEEKS,1_2_WEEKS,N,6_12_MONTHS,AFP MEASLES YELLOW_FEVER NNT,NA,N,-7.377393,20.834097,725.799988,5.000,None,100.0,None,-1.0,None,None,2.017090e+11,428.0,None,866.0,-7.377344,20.834137,44.0,2017.0,POINT Z (20.83410 -7.37739 0.00000)


In [5]:
# Define some variables
country_col = 'countries'
name_col = 'name_of_facility_visited'
type_col = 'type_of_facility_visited'

In [6]:
# Name of DRC is different from WHO dataset
priority_countries = ['South Sudan', 'Mozambique', 'Namibia', 'Nigeria', 'Zambia',
                      'Sierra Leone', 'Ghana',  'Burkina Faso', 'Ethiopia', 'Somalia', 
                      'Rwanda', 'Kenya', 'Zimbabwe', 'République Démocratique Du Congo']

# 1. What is the list of fields?

In [7]:
print("The fields are", df.columns)

The fields are Index(['starttime', 'endtime', 'today', 'deviceid', 'date_of_visit', 'ist',
       'countries', 'states', 'districts', 'wards', 'designation',
       'name_of_facility_visited', 'name_of_facility_visited_other',
       'type_of_facility_visited', 'priority_level', 'facility_fp_acs',
       'joint_supervision', 'supervised_by_dsno', 'supervised_by_who',
       'current_feed_back', 'surveillance_fp_training', 'case_def_copy_copy1',
       'facility_fp_acs_copy', 'physiotherapy_unit', 'long', 'lat', 'F27',
       'F28', 'F29', 'F_index', 'F_parent_table_name', 'F_parent_index',
       'F_tags', 'F_notes', 'F_version', 'F_duration', 'F_submitted_by',
       'F_xform_id', 'longitude', 'latitude', 'Weeknumbers',
       'YearofActiveSurv', 'geometry'],
      dtype='object')


# 2. What does each describe?

- `starttime`: start time.
- `endtime`: end time.
- `today`: today's date and day of week
- `deviceid`
- `date_of_visit`
- `ist`: region with respect to the continent, unique values include West, Central and East South.
- `countries`
- `states` 
- `districts`
- `wards`: administrative division.
- `designation`: title.

- `name_of_facility_visited`
- `name_of_facility_visited_other`: another name the facility might have.
- `type_of_facility_visited`
- `priority_level` 
- `facility_fp_acs`: focal person access (inferred), values could be 'Y', 'NA', 'N' or missing.
- `joint_supervision`: whether the facility is jointly supervised. 'Y' or 'N'.
- `supervised_by_dsno`: last time the facility was supervised by DSNO. Values could include 'Not in the past 1 year', 'more than 6 months', 'more than 3 months' and so on.
- `supervised_by_who`: last time the facility was supervised by DSNO. Values could include 'Not in the past 1 year', 'more than 6 months', 'more than 3 months' and so on.

- `current_feed_back`: 'Y' or 'N'
- `surveillance_fp_training`: when training is given to focal person or there is no training or focal person is not assigned.
- `case_def_copy_copy1`: ?
- `facility_fp_acs_copy`: ?
- `physiotherapy_unit`: whether there is a physiotherapy unit.
- `long`: longitude
- `lat`: latitude.
- `F27`: ?
- `F28`: ?
- `F29`: almost 100% missing 
- `F_index` 
- `F_parent_table_name`: almost 100% missing 
- `F_parent_index`
- `F_tags`: almost 100% missing 
- `F_notes`: almost 100% missing 
- `F_version`
- `F_duration`
- `F_submitted_by`: almost 100% missing 
- `F_xform_id`
- `longitude` 
- `latitude` 
- `Weeknumbers`: week number in the year
- `YearofActiveSurv`: year
- `geometry`: contains the geometric object

In [8]:
# Examine unique values of certain columns
cols = ['ist', 'facility_fp_acs', 'joint_supervision', 'supervised_by_dsno',
       'current_feed_back', 'surveillance_fp_training', 'case_def_copy_copy1',
       'facility_fp_acs_copy', 'physiotherapy_unit']
for col in cols:
    n_unique = df[col].nunique()
    
    if n_unique < 20:
        print("Unique values of column", col + ':', df[col].unique())
        print('\n')
    
    else:
        print("Column", col, "has too many unique values.")
        print("Example values:", df[col].unique()[:5])
        print('\n')

Unique values of column ist: ['IST_WEST' 'IST_CENTRAL' 'IST_EAST_SOUTH']


Unique values of column facility_fp_acs: ['Y' 'NA' 'N' None]


Unique values of column joint_supervision: ['N' 'Y' None]


Unique values of column supervised_by_dsno: ['NOT_IN_THE_PAST_1_YEAR' '1_2_WEEKS' '1_MONTH' 'LESS_THAN_1_WEEK'
 '2_3_MONTHS' 'MORE_THAN_6_MONTHS' 'MORE_THAN_3_MONTHS' None 'DN']


Unique values of column current_feed_back: ['N' 'Y' None]


Unique values of column surveillance_fp_training: ['6_MONTHS' 'MORE_THAN_2_YRS' '6_12_MONTHS' '1_2_YRS' 'NOT_TRAINED'
 'NO_FOCAL_PERSON_ASSIGNED' None 'UNABLE_TO_ASSESS']


Column case_def_copy_copy1 has too many unique values.
Example values: ['AFP MEASLES CEREBROSPINAL_MENINGITIS NNT'
 'AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGITIS NNT'
 'AFP MEASLES YELLOW_FEVER NNT' 'AFP MEASLES' 'AFP']


Unique values of column facility_fp_acs_copy: ['Y' 'NA' 'N' None 'N_A' 'updated' 'nocase' 'notupdated' 'no']


Unique values of column physiotherapy_unit: ['N' 'Y

# 3. Are they all populated?

Most columns have less than 1% missing values. Some columns have more than 50% missing and a few columns have almost 100% missing values.

In [9]:
print("NA values by column")
pd.DataFrame({'count':df.isna().sum(), 'percentage':round(df.isna().sum() / df.shape[0],3) * 100})

NA values by column


,count,percentage
starttime,17,0.0
endtime,96,0.0
today,4,0.0
deviceid,2041,0.7
date_of_visit,1,0.0
ist,0,0.0
countries,0,0.0
states,276,0.1
districts,643,0.2
wards,7409,2.4


# 4. Is there information on when the data was collected?  If so, what is the date range?

Time related columns are `starttime`, `endtime`, `today`, `date_of_visit`, `Weeknumbers`, `YearofActiveSurv`.

`starttime`, `endtime`, `today`, `date_of_visit` all have strange values. The start and end dates are listed below. `Weeknumbers`, `YearofActiveSurv` seem to be within a reasonable range. Some data cleaning is done to choose one of `today`, `date_of_visit` as the correct date. If both are incorrect, the date is left as na.

In [10]:
# For selecting the time related columns 
cols = ['starttime','endtime','today_dt', 'date_of_visit_dt','Weeknumbers','YearofActiveSurv', 
        'today_wk_num', 'dov_wk_num']

## `today`

In [11]:
# change time related columns to datetime format
df['today_dt'] = pd.to_datetime(df['today'])
print('Column: today')
print("start:", df['today_dt'].min().date(), 
      "\nend:", df['today_dt'].max().date())

Column: today
start: 1970-01-01 
end: 2028-05-17


In [12]:
df[df['today_dt']=='1970-01-01'].head()

,starttime,endtime,today,deviceid,date_of_visit,ist,countries,states,districts,wards,designation,name_of_facility_visited,name_of_facility_visited_other,type_of_facility_visited,priority_level,facility_fp_acs,joint_supervision,supervised_by_dsno,supervised_by_who,current_feed_back,surveillance_fp_training,case_def_copy_copy1,facility_fp_acs_copy,physiotherapy_unit,long,lat,F27,F28,F29,F_index,F_parent_table_name,F_parent_index,F_tags,F_notes,F_version,F_duration,F_submitted_by,F_xform_id,longitude,latitude,Weeknumbers,YearofActiveSurv,geometry,today_dt
275296,01/01/1970 2:41:48 am +02:00,27/09/2017 1:36:10 pm +02:00,"Thursday, 1 January 1970",8.678190e+14,"Tuesday, 26 September 2017",IST_CENTRAL,RÉPUBLIQUE DÉMOCRATIQUE DU CONGO,HAUT-KATANGA,TSHAMILEMBA,QUARTIER INDUSTRIEL,OTHERS,CS VIRGINIKA,None,PRIVATE_FACILITY,H,N,Y,1_MONTH,NOT_IN_THE_PAST_1_YEAR,Y,6_MONTHS,AFP MEASLES YELLOW_FEVER VHF CHOLERA NNT,N,N,-11.647236,27.492217,1267.0,4.000,None,117.0,None,-1.0,None,None,2.017090e+11,1.506510e+09,None,NaN,-11.647318,27.492241,40.0,2017.0,POINT Z (27.49222 -11.64724 0.00000),1970-01-01
275856,01/01/1970 1:50:56 am +01:00,17/10/2018 6:33:13 pm +01:00,"Thursday, 1 January 1970",8.678190e+14,"Monday, 15 October 2018",IST_CENTRAL,RÉPUBLIQUE DÉMOCRATIQUE DU CONGO,KINSHASA,BUMBU,Matadi 1,REGIONAL_RI_OFFICER,Cs. NGEBA,None,CSI,L,Y,N,1_MONTH,NOT_IN_THE_PAST_1_YEAR,Y,NOT_TRAINED,NONE,N,N,-4.315936,15.316878,279.0,13.000,None,3632.0,None,-1.0,None,None,2.018070e+11,1.539795e+09,None,841.0,-4.375504,15.285431,48.0,2018.0,POINT Z (15.31688 -4.31594 0.00000),1970-01-01
279598,01/01/1970 5:58:13 am +01:00,02/02/2019 11:46:01 pm +01:00,"Thursday, 1 January 1970",8.678190e+14,"Thursday, 3 January 2019",IST_CENTRAL,RÉPUBLIQUE DÉMOCRATIQUE DU CONGO,KINSHASA,BUMBU,Dipiya,REGIONAL_RI_OFFICER,TWUISANA,None,CSI,M,Y,N,1_MONTH,NOT_IN_THE_PAST_1_YEAR,Y,MORE_THAN_2_YRS,AFP MEASLES YELLOW_FEVER VHF CHOLERA NNT,N,N,-4.322421,15.315941,283.0,5.000,None,4228.0,None,-1.0,None,None,2.018070e+11,1.549130e+09,None,841.0,-4.322451,15.315890,6.0,2019.0,POINT Z (15.31594 -4.32242 0.00000),1970-01-01
279625,01/01/1970 1:28:38 pm +01:00,27/02/2019 10:14:39 am +01:00,"Thursday, 1 January 1970",8.678190e+14,"Wednesday, 27 February 2019",IST_CENTRAL,RÉPUBLIQUE DÉMOCRATIQUE DU CONGO,KINSHASA,KINTAMBO,TSHINKELA,REGIONAL_RI_OFFICER,KAMZ CLINIQUE,None,PRIVATE_FACILITY,M,Y,Y,1_2_WEEKS,1_MONTH,Y,MORE_THAN_2_YRS,AFP MEASLES YELLOW_FEVER VHF CHOLERA NNT,N,N,-4.340013,15.268305,0.0,1000.000,None,4315.0,None,-1.0,None,None,2.017100e+11,1.551214e+09,None,841.0,-4.339702,15.268305,9.0,2019.0,POINT Z (15.26830 -4.34001 0.00000),1970-01-01
279884,01/01/1970 1:02:07 am +01:00,10/05/2019 9:05:23 am +01:00,"Thursday, 1 January 1970",8.678190e+14,"Friday, 10 May 2019",IST_CENTRAL,RÉPUBLIQUE DÉMOCRATIQUE DU CONGO,KINSHASA,KINTAMBO,TSHINKELA,REGIONAL_RI_OFFICER,NEW KAMZ CLINIQUE,None,PRIVATE_FACILITY,M,Y,Y,1_MONTH,2_3_MONTHS,Y,MORE_THAN_2_YRS,AFP MEASLES YELLOW_FEVER VHF CHOLERA NNT,N,N,-4.440765,15.287152,486.0,3.326,None,5059.0,None,-1.0,None,None,2.017100e+11,1.557475e+09,None,841.0,-4.440651,15.287133,19.0,2019.0,POINT Z (15.28715 -4.44076 0.00000),1970-01-01


In [13]:
df[df['today_dt']=='2028-05-17'].head()

,starttime,endtime,today,deviceid,date_of_visit,ist,countries,states,districts,wards,designation,name_of_facility_visited,name_of_facility_visited_other,type_of_facility_visited,priority_level,facility_fp_acs,joint_supervision,supervised_by_dsno,supervised_by_who,current_feed_back,surveillance_fp_training,case_def_copy_copy1,facility_fp_acs_copy,physiotherapy_unit,long,lat,F27,F28,F29,F_index,F_parent_table_name,F_parent_index,F_tags,F_notes,F_version,F_duration,F_submitted_by,F_xform_id,longitude,latitude,Weeknumbers,YearofActiveSurv,geometry,today_dt
300235,17/05/2028 2:28:11 pm +03:00,20/05/2028 2:58:57 am +03:00,"Wednesday, 17 May 2028",8.694580e+14,"Wednesday, 17 May 2028",IST_EAST_SOUTH,TANZANIA,KATAVI,NSIMBO DC,Katumba,DIVO,Kalungu,None,DISPENSARY,L,N,N,2_3_MONTHS,2_3_MONTHS,Y,NOT_TRAINED,None,NA,N,-6.30699,31.31896,1207.6,10.0,None,1.0,None,-1.0,None,None,2.017100e+11,217845.0,None,951.0,-6.299743,31.318835,21.0,2018.0,POINT Z (31.31896 -6.30699 0.00000),2028-05-17


## `date_of_visit`

In [14]:
# change time related columns to datetime format
df['date_of_visit_dt'] = pd.to_datetime(df['date_of_visit'])
print('Column: date_of_visit')
print("start:", df['date_of_visit_dt'].min().date(), 
      "\nend:", df['date_of_visit_dt'].max().date())

Column: date_of_visit
start: 1900-02-11 
end: 2029-04-05


In [15]:
df[df['date_of_visit_dt']=='1900-02-11'].head()

,starttime,endtime,today,deviceid,date_of_visit,ist,countries,states,districts,wards,designation,name_of_facility_visited,name_of_facility_visited_other,type_of_facility_visited,priority_level,facility_fp_acs,joint_supervision,supervised_by_dsno,supervised_by_who,current_feed_back,surveillance_fp_training,case_def_copy_copy1,facility_fp_acs_copy,physiotherapy_unit,long,lat,F27,F28,F29,F_index,F_parent_table_name,F_parent_index,F_tags,F_notes,F_version,F_duration,F_submitted_by,F_xform_id,longitude,latitude,Weeknumbers,YearofActiveSurv,geometry,today_dt,date_of_visit_dt
290557,2018-02-12T13:07:05.646Z,2018-02-12T14:06:22.243Z,"Monday, 12 February 2018",8.678190e+14,"Sunday, 11 February 1900",IST_WEST,SIERRA LEONE,Eastern,KONO,Nimiyama,WCO_SURVEILLANCE_OFFICER,Ngotown CHP,None,PHC_CENTER,M,Y,Y,LESS_THAN_1_WEEK,MORE_THAN_6_MONTHS,N,MORE_THAN_2_YRS,AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGI...,N,N,8.573498,-11.307923,353.0,5.0,None,611.0,None,-1.0,None,None,2.017120e+11,3557.0,None,484.0,8.573513,-11.307756,8.0,2018.0,POINT Z (-11.30792 8.57350 0.00000),2018-02-12,1900-02-11


## `Weeknumbers`

In [16]:
df['Weeknumbers_new'] = df['Weeknumbers'].astype(int)
print('Column: Weeknumbers')
print('min value:', min(df['Weeknumbers_new']))
print('max value:', max(df['Weeknumbers_new']))

Column: Weeknumbers
min value: 1
max value: 53


## `YearofActiveSurv`

In [17]:
df['YearofActiveSurv_new'] = df['YearofActiveSurv'].astype(int)
print('Column: YearofActiveSurv')
print("start:", df['YearofActiveSurv_new'].min(), 
      "\nend:", df['YearofActiveSurv_new'].max())

Column: YearofActiveSurv
start: 2017 
end: 2020


## date cleaning

In [18]:
# create a new 'date' column
dates = []
for idx, row in df.iterrows():
    dov = row['date_of_visit_dt']
    today = row['today_dt']
    
    # if 'date_of_visit' is within the time frame, take 'date_of_visit' as date
    if dov >= pd.to_datetime('2017-01-01') and dov < pd.to_datetime('2020-07-01'):
        dates.append(dov)
    
    # if 'today' is within the time frame, take 'today' as date
    elif today >= pd.to_datetime('2017-01-01') and today < pd.to_datetime('2020-07-01'):
        dates.append(today)
    
    # otherwise, date is na
    else:
        dates.append(np.nan)
        
df['date'] = dates

In [19]:
print("Number of NAs as a result:", df['date'].isna().sum())

Number of NAs as a result: 50


In [20]:
print("start date:", df['date'].min().date(), 
      "\nend date:", df['date'].max().date())

start date: 2017-01-01 
end date: 2020-06-18


In [21]:
wk_nums = df['date'].dt.week
print("Number of values where week number differs by more than 1:", sum(np.abs(wk_nums - df['Weeknumbers'])>1))

Number of values where week number differs by more than 1: 38264


In [23]:
years = df['date'].dt.year
print("Number of values where year number differs by more than 1:", sum(years - df['YearofActiveSurv']>1))

Number of values where year number differs by more than 1: 0


In [24]:
# reformat the country column
df[country_col+'_title'] = df[country_col].str.title()
country_col += '_title'

print("date range by country")
df[df[country_col].isin(priority_countries)].groupby(country_col) \
.agg(start=('date', 'min'), end=('date', 'max'))

date range by country


,start,end
countries_title,,
Burkina Faso,2017-12-12,2020-05-25
Ethiopia,2019-11-29,2020-05-25
Ghana,2017-10-10,2020-05-25
Kenya,2018-08-06,2020-06-06
Mozambique,2017-12-04,2020-04-08
Namibia,2018-04-18,2020-05-15
Nigeria,2017-01-01,2020-05-17
Rwanda,2018-04-17,2020-02-27
République Démocratique Du Congo,2017-01-15,2020-05-25


# 5. How many countries does the dataset cover?

In [25]:
print("Number of countries covered:", df[country_col].nunique())

Number of countries covered: 47


# 6. What are those countries?

In [26]:
print("Countries covered:", df[country_col].unique())

Countries covered: ['Algeria' 'Angola' 'Benin' 'Botswana' 'Burkina Faso' 'Burundi' 'Cameroon'
 'Central_African_Republic' 'Chad' 'Congo' "Cote D'Ivoire"
 'Equatorial Guinea' 'Eritrea' 'Eswatini' 'Ethiopia' 'Gabon' 'Gambia'
 'Ghana' 'Guinea' 'Guinea-Bissau' 'Kenya' 'Lesotho' 'Liberia' 'Madagascar'
 'Malawi' 'Mali' 'Mauritania' 'Mauritius' 'Mozambique' 'Namibia' 'Niger'
 'Nigeria' 'République Démocratique Du Congo' 'Rwanda'
 'Sao Tome And Principe' 'Senegal' 'Seychelles' 'Sierra' 'Sierra Leone'
 'South Africa' 'South Sudan' 'Swaziland' 'Tanzania' 'Togo' 'Uganda'
 'Zambia' 'Zimbabwe']


# 7. How many data points have been collected overall?

In [27]:
print("Number of data points collected:", df.shape[0])

Number of data points collected: 308327


# 8. How many of the facility names are empty, both null or no text?

Observation: Some facility names only have single character.

In [28]:
print("Number of null values:", df[name_col].isna().sum())

Number of null values: 636


In [29]:
print("Number of no text values:", sum(df[name_col] == ""))

Number of no text values: 0


In [30]:
print("Number of data points:", df.shape[0])
print("Number of unique facility names:", df[name_col].nunique())

Number of data points: 308327
Number of unique facility names: 108427


In [31]:
name_len = name_col + '_len'
df[name_len] = df[name_col].str.len()

In [32]:
# facilities with shortest names
df.sort_values(by=name_len).head()

,starttime,endtime,today,deviceid,date_of_visit,ist,countries,states,districts,wards,designation,name_of_facility_visited,name_of_facility_visited_other,type_of_facility_visited,priority_level,facility_fp_acs,joint_supervision,supervised_by_dsno,supervised_by_who,current_feed_back,surveillance_fp_training,case_def_copy_copy1,facility_fp_acs_copy,physiotherapy_unit,long,lat,F27,F28,F29,F_index,F_parent_table_name,F_parent_index,F_tags,F_notes,F_version,F_duration,F_submitted_by,F_xform_id,longitude,latitude,Weeknumbers,YearofActiveSurv,geometry,today_dt,date_of_visit_dt,Weeknumbers_new,YearofActiveSurv_new,date,countries_title,name_of_facility_visited_len
188721,25/07/2018 2:36:13 pm +01:00,28/08/2018 5:00:44 pm +01:00,"Wednesday, 25 July 2018",3.591440e+14,"Thursday, 28 June 2018",IST_WEST,NIGERIA,Adamawa,Demsa,Bille,SC,g,None,SPIRIT_HEALER,OTHER_NON_FS,NA,N,NOT_IN_THE_PAST_1_YEAR,NOT_IN_THE_PAST_1_YEAR,None,NO_FOCAL_PERSON_ASSIGNED,NOT_APPLICABLE,None,None,10.419273,8.678660,810.400000,6.8,None,103825.0,None,-1.0,None,None,2.017080e+11,2946271.0,None,562.0,10.407680,8.690827,35.0,2018.0,POINT Z (8.67866 10.41927 0.00000),2018-07-25,2018-06-28,35,2018,2018-06-28,Nigeria,1.0
228294,04/03/2020 11:28:51 am +01:00,04/03/2020 12:22:16 pm +01:00,"Wednesday, 4 March 2020",3.589820e+14,"Wednesday, 4 March 2020",IST_WEST,NIGERIA,Borno,Shani,Bargu,LGAF,Y,None,PHC_CENTER,H,Y,Y,1_2_WEEKS,1_2_WEEKS,None,6_12_MONTHS,NONE,updated,None,10.087350,11.962473,252.288101,4.0,None,200834.0,None,-1.0,None,None,2.019090e+11,3206.0,None,562.0,10.087342,11.962503,10.0,2020.0,POINT Z (11.96247 10.08735 0.00000),2020-03-04,2020-03-04,10,2020,2020-03-04,Nigeria,1.0
248271,14/06/2019 2:34:08 pm +01:00,19/06/2019 9:44:11 am +01:00,"Friday, 14 June 2019",3.536450e+14,"Friday, 14 June 2019",IST_WEST,NIGERIA,Lagos,Badagry,AJARA TOPA,ME,A,None,PHC_CENTER,M,Y,Y,1_MONTH,LESS_THAN_1_WEEK,None,6_MONTHS,AFP MEASLES,updated,None,6.581119,3.341494,46.000000,16.0,None,155465.0,None,-1.0,None,None,2.019060e+11,414603.0,None,562.0,6.434697,2.890666,25.0,2019.0,POINT Z (3.34149 6.58112 0.00000),2019-06-14,2019-06-14,25,2019,2019-06-14,Nigeria,1.0
189670,02/06/2017 9:41:46 am +01:00,02/06/2017 9:48:49 am +01:00,"Friday, 2 June 2017",3.539930e+14,"Friday, 2 June 2017",IST_WEST,NIGERIA,Kwara,Pategi,kpada II,LGAF,.,None,TRAD_SPIRIT_HEALER,OTHER_NON_FS,NA,N,MORE_THAN_3_MONTHS,2_3_MONTHS,None,NO_FOCAL_PERSON_ASSIGNED,NOT_APPLICABLE,None,None,8.716448,5.868016,130.235882,32.0,None,2031.0,None,-1.0,None,None,2.017050e+11,423.0,None,NaN,8.716712,5.868207,24.0,2017.0,POINT Z (5.86802 8.71645 0.00000),2017-06-02,2017-06-02,24,2017,2017-06-02,Nigeria,1.0
25664,16/09/2019 12:05:22 pm +01:00,16/09/2019 12:50:04 pm +01:00,"Monday, 16 September 2019",3.596180e+14,"Monday, 16 September 2019",IST_CENTRAL,CONGO,POINTE NOIRE,TCHIAMBA NZASSI,CSI PMAE TCHIAMBA- NZASSI,STOP_TEAM,R,None,CSI,H,Y,Y,1_MONTH,1_MONTH,Y,6_12_MONTHS,AFP MEASLES YELLOW_FEVER NONE,N,N,-4.987623,12.045223,-99.900000,16.2,None,1227.0,None,-1.0,None,None,2.018070e+11,2682.0,None,1172.0,-4.987760,12.045483,41.0,2019.0,POINT Z (12.04522 -4.98762 0.00000),2019-09-16,2019-09-16,41,2019,2019-09-16,Congo,1.0


# 9. How many data points have been collected for each country?

Somalia is not covered by the dataset.

Also, around 67% of the data points are in Nigeria.

In [35]:
counts_by_country = df[country_col].value_counts()
print("Raw counts by country:")
print(counts_by_country)

Raw counts by country:
Nigeria                             208655
République Démocratique Du Congo     13518
Mali                                 10489
Chad                                  9900
Cameroon                              7779
South Sudan                           5279
Liberia                               4348
Zambia                                3731
Madagascar                            3657
Gabon                                 3502
Ghana                                 3089
Angola                                2968
Tanzania                              2675
Ethiopia                              2487
Uganda                                2329
Sierra Leone                          2300
Malawi                                2152
Niger                                 2010
Central_African_Republic              1729
Congo                                 1688
Benin                                 1607
Kenya                                 1552
Mozambique                     

In [34]:
# Number of data points for priority countries
print('Raw counts for priority countries only:')
counts_by_country[counts_by_country.index.isin(priority_countries)].sort_values(ascending=False)

Raw counts for priority countries only:


Nigeria                             208655
République Démocratique Du Congo     13518
South Sudan                           5279
Zambia                                3731
Ghana                                 3089
Ethiopia                              2487
Sierra Leone                          2300
Kenya                                 1552
Mozambique                            1430
Namibia                                515
Burkina Faso                           380
Zimbabwe                               287
Rwanda                                 185
Name: countries_title, dtype: int64

In [36]:
print("Counts for priority countries based on number of unique facility names:")
df[df[country_col].isin(priority_countries)].groupby(country_col)\
.agg(count=(name_col, 'nunique')).sort_values('count' ,ascending=False)

Counts for priority countries based on number of unique facility names:


,count
countries_title,
Nigeria,63790
République Démocratique Du Congo,8113
Ghana,2027
Sierra Leone,1434
South Sudan,1144
Zambia,1012
Kenya,899
Ethiopia,875
Mozambique,377


In [37]:
s_priority_countries = pd.Series(priority_countries)
print("Priority countries not covered: ")
print(s_priority_countries[~s_priority_countries.isin(counts_by_country.index)].values)

Priority countries not covered: 
['Somalia']


# 10. What are the “types” used for each priority country(listed below)? 
Each health Facility is assigned a type such as Hospital, Health Facility, etc. Please list the unique types and count for each.

In [38]:
for country in priority_countries:
    if country in df[country_col].unique():
    
        df_ctr = df[df[country_col]==country]
        print(country + ',', "Number of unique facility types:", df_ctr[type_col].nunique())
        print(df_ctr[type_col].value_counts())
        print('\n')
    
    else:
        print(country, "not covered by the dataset")
        print('\n')

South Sudan, Number of unique facility types: 14
PHCC                      2409
PHCU                      2124
COUNTY_HOSP                275
STATE_HOSP                 173
TEACHING_HOSP              105
PRIVATE_FACILITY           102
TRAD_HEALER                 35
GENERAL_HOSP                27
TRAD_BIRTH_ATTENDANT        12
OTHER_NON_ORTHORDOX_HC       6
SPIRIT_HEALER                5
FAITH_BASED                  3
KINESITHERAPIE               2
MILITARY                     1
Name: type_of_facility_visited, dtype: int64


Mozambique, Number of unique facility types: 10
HEALTH_FACILITY     1205
HEALTH_POST           95
DISTRICT_HOSP         59
RURAL_HOSPITAL        23
GENERAL_HOSP          18
CENTRAL_HOSPITAL      16
others                 7
SPIRITUAL_HEALER       3
PROVINCIAL_HOSP        2
PRIVATE_FACILITY       2
Name: type_of_facility_visited, dtype: int64


Namibia, Number of unique facility types: 7
CLINICS             370
HEALTH_CENTRES       70
DISTRICT_HOSP        56
REGIONAL_

# 11. Make a map of the dataset, how does the data look? Does it all fall within Africa?

Since `geometry` has some missing values, rows with `geometry` missing are dropped. 

Since trying to plot all data points really slows things down, 1000 data points are randomly sampled and plotted on the map. The process is repeated by changing the random state to view different data points.

It looks like the sampled data points all fall within Africa. There are a few points that might be on some islands near the African continent.

In [40]:
# Re-project to WGS84
df_map = df.dropna(subset=['geometry'])

In [41]:
df_map['geometry'] = df_map['geometry'].to_crs(epsg=4326)

C:\Users\DUANYUEYUN\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
# Define some global variables
CENTER_POINTS = {}
# overall
CENTER_POINTS['Overall'] = [0.8560,21.5125]
# country with lowest number of observations, used to test things out
CENTER_POINTS['Somalia'] = [6.8864,45.1807]
# Countries with the largest number of observations
CENTER_POINTS['Ghana'] = [8.1813, -1.1766]

CENTER_POINTS['Namibia'] = [-22.5644, 17.0718]

CENTER_POINTS['Nigeria'] = [9.0584, 7.3895]

In [43]:
def make_map(zoom_start, sample_size, random_state, df):
    m = folium.Map(location=CENTER_POINTS['Overall'], 
                   zoom_start=zoom_start, control_scale=True)

    # Create a Clustered map where points are clustered
    marker_cluster = MarkerCluster().add_to(m)
    # Create health facilities as points on top of the map
    for idx, row in df.sample(sample_size, random_state=random_state).iterrows():
        # Get lat and lon of points
        lon = row['geometry'].x
        lat = row['geometry'].y

        # Add marker to the map
        folium.Circle(location=[lat, lon], 
                      radius=2).add_to(m) 
    return m

In [44]:
#make_map(zoom_start=2, sample_size=1000, random_state=25, df=df_map)

In [45]:
#make_map(zoom_start=3, sample_size=1000, random_state=50, df=df_map)

In [46]:
make_map(zoom_start=3, sample_size=1000, random_state=75, df=df_map)

In [60]:
#m = make_map(zoom_start=3, sample_size=1000, random_state=25, df=df_map)
#outfp = r'ISS_map_marker.html'
#m.save(outfp)

# 12. Zooming into the map do you see multiple data points collected for the same location?

Yes. A map with location markers and a screenshot with overlapping data points are included below. From the screenshot, it looks like the two health facilities are plotted at the same location. Also they have the same name.

In [67]:
# Use Nigeria as the center point since that's the area where most health facilities are concentrated
m = folium.Map(location=CENTER_POINTS['Nigeria'], 
                zoom_start=5, control_scale=True)

# Create a Clustered map where points are clustered
marker_cluster = MarkerCluster().add_to(m)
# Create health facilities as points on top of the map
for idx, row in df_map.sample(1000, random_state=25).iterrows():
        # Get lat and lon of points
    lon = row['geometry'].x
    lat = row['geometry'].y
    
    facility_name = row[name_col]

    # Add marker to the map
    folium.Marker(location=[lat, lon], popup=facility_name,
                      radius=2).add_to(m) 
m

!['overlapping points'](maps/ISS_12.png)

In [68]:
# Rows with the health facility name shown above
df[df[name_col]=='MPHC GAURAKA']

,starttime,endtime,today,deviceid,date_of_visit,ist,countries,states,districts,wards,designation,name_of_facility_visited,name_of_facility_visited_other,type_of_facility_visited,priority_level,facility_fp_acs,joint_supervision,supervised_by_dsno,supervised_by_who,current_feed_back,surveillance_fp_training,case_def_copy_copy1,facility_fp_acs_copy,physiotherapy_unit,long,lat,F27,F28,F29,F_index,F_parent_table_name,F_parent_index,F_tags,F_notes,F_version,F_duration,F_submitted_by,F_xform_id,longitude,latitude,Weeknumbers,YearofActiveSurv,geometry,date_of_visit_dt,today_dt,Weeknumbers_new,YearofActiveSurv_new,name_of_facility_visited_len,countries_cap,countries_title
66670,18/10/2017 10:29:23 am +01:00,18/10/2017 11:57:50 am +01:00,"Wednesday, 18 October 2017",3.566020e+14,"Wednesday, 18 October 2017",IST_WEST,NIGERIA,Niger,Tafa,Zuma East,LGAF,MPHC GAURAKA,None,PHC_CENTER,H,Y,N,1_2_WEEKS,1_2_WEEKS,None,6_MONTHS,AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGI...,updated,None,9.178733,7.220148,475.7,9.400,None,32847.0,None,-1.0,None,None,2.017050e+11,5308.0,None,562.0,9.177725,7.224138,43.0,2017.0,POINT Z (7.22015 9.17873 0.00000),2017-10-18,2017-10-18,43,2017,12.0,Nigeria,Nigeria
66807,25/10/2017 11:21:04 am +01:00,26/10/2017 12:11:37 pm +01:00,"Wednesday, 25 October 2017",3.566020e+14,"Wednesday, 25 October 2017",IST_WEST,NIGERIA,Niger,Tafa,Zuma East,LGAF,MPHC GAURAKA,None,PHC_CENTER,H,Y,N,1_2_WEEKS,1_2_WEEKS,None,6_MONTHS,AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGI...,updated,None,9.178868,7.220665,457.5,9.600,None,34569.0,None,-1.0,None,None,2.017050e+11,89433.0,None,562.0,9.178602,7.220080,44.0,2017.0,POINT Z (7.22067 9.17887 0.00000),2017-10-25,2017-10-25,44,2017,12.0,Nigeria,Nigeria
67123,13/11/2017 1:00:29 pm +01:00,13/11/2017 2:26:12 pm +01:00,"Monday, 13 November 2017",3.566020e+14,"Monday, 13 November 2017",IST_WEST,NIGERIA,Niger,Tafa,Zuma East,LGAF,MPHC GAURAKA,None,PHC_CENTER,H,Y,N,1_2_WEEKS,1_2_WEEKS,None,6_MONTHS,AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGI...,updated,None,9.179237,7.220770,101.8,9.900,None,38496.0,None,-1.0,None,None,2.017050e+11,5142.0,None,562.0,9.178580,7.220023,47.0,2017.0,POINT Z (7.22077 9.17924 0.00000),2017-11-13,2017-11-13,47,2017,12.0,Nigeria,Nigeria
67560,04/12/2017 12:50:52 pm +01:00,04/12/2017 2:02:22 pm +01:00,"Monday, 4 December 2017",3.566020e+14,"Monday, 4 December 2017",IST_WEST,NIGERIA,Niger,Tafa,Zuma East,LGAF,MPHC GAURAKA,None,PHC_CENTER,H,Y,N,1_MONTH,1_2_WEEKS,None,6_MONTHS,AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGI...,updated,None,9.178590,7.220190,466.3,9.000,None,43965.0,None,-1.0,None,None,2.017050e+11,4291.0,None,562.0,9.178625,7.220085,50.0,2017.0,POINT Z (7.22019 9.17859 0.00000),2017-12-04,2017-12-04,50,2017,12.0,Nigeria,Nigeria
67744,13/12/2017 10:25:50 am +01:00,13/12/2017 11:42:49 am +01:00,"Wednesday, 13 December 2017",3.566020e+14,"Wednesday, 13 December 2017",IST_WEST,NIGERIA,Niger,Tafa,Zuma East,LGAF,MPHC GAURAKA,None,PHC_CENTER,H,Y,Y,1_2_WEEKS,1_2_WEEKS,None,6_MONTHS,AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGI...,updated,None,9.178553,7.219563,280.0,9.100,None,46039.0,None,-1.0,None,None,2.017050e+11,4619.0,None,562.0,9.178612,7.219950,51.0,2017.0,POINT Z (7.21956 9.17855 0.00000),2017-12-13,2017-12-13,51,2017,12.0,Nigeria,Nigeria
68204,09/01/2018 9:51:14 pm +01:00,09/01/2018 10:53:27 pm +01:00,"Tuesday, 9 January 2018",3.566020e+14,"Wednesday, 10 January 2018",IST_WEST,NIGERIA,Niger,Tafa,Zuma East,LGAF,MPHC GAURAKA,None,PHC_CENTER,H,Y,N,1_2_WEEKS,1_2_WEEKS,None,6_12_MONTHS,AFP MEASLES YELLOW_FEVER CEREBROSPINAL_MENINGI...,updated,None,9.178628,7.220473,360.9,8.300,None,51385.0,None,-1.0,None,None,2.017050e+11,3733.0,None,562.0,9.178685,7.220142,2.0,2018.0,POINT Z (7.22047 9.17863 0.00000),2018-01-10,2018-01-09,2,2018,12.0,Nigeria,Nigeria
68402,18/01/2018 12:06:48 am +01:00,18/01/2018 1:24:05 am +01:00,"Thursday, 18 January 2018",3.566020e+14,"Thursday, 18 January 2018",IST_WEST,NIGERIA,Niger,Tafa,Zuma East,LGAF,MPHC GAURAKA,None,PHC_CENTER,H,Y,Y,1_2_W

# 13. What other observation can be made when looking at the map?

Some of the health facilities are located near the border, which is a little surprising.

![](maps/ISS_13.png)